API für DinoProject

Methoden:

POST /game: Erstellt eine neue Session und gibt einen Redirect zurück

GET /highscore/{id} : Liefert zu einer Session-ID den Highscore zurück <br/>
POST /highscore/{id} : Aktualisiert bei einer Session den Highscore oder legt ihn neu an

In [1]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_cors import CORS
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify, redirect
import optparse
import mysql.connector
import uuid

app = Flask(__name__)
cors = CORS(app, resources={r"*": {"origins": "*"}}) # CORS
api = Api(app)


# Datenbank konfigurieren und Verbindung herstellen
mydb = mysql.connector.connect(
    host = "webengineering.ins.hs-anhalt.de",
    port = 32192,
    user = "root",
    passwd = "ga4wm19",
    database = "dinoProject"
)

mycursor = mydb.cursor()
mydb.disconnect() # Vorläufig Verbindung schließen

In [2]:
# Klasse um eine neue Session zu erstellen
class createSession(Resource):
    def post(self):
        sessionID = str(uuid.uuid4().hex)
        return redirect('http://localhost:8080/game/' + sessionID, code=303)

# Klasse für Updaten und Abfragen eines spezifischen Highscores
class getOrAddHighscore(Resource):
    def get(self, id):
        sql = "SELECT * FROM dinoProject.highscoreTable WHERE sessionID = '" + id + "'"
        mydb.reconnect(attempts = 1 , delay = 0)
        mycursor.execute(sql)
        dbResult = mycursor.fetchall()
        
        if len(dbResult) == 1 :
            session = dbResult[0]
            return jsonify({'sessionID': session[0], 'highscore': session[1]})
        else:
            return jsonify({'status': 'none or more than one highscore found', 'statuscode': 409})
        
    def post(self, id):
        if not request.json or not "highscore" in request.json or not "sessionID" in request.json:
            return jsonify({'status': 'could not update highscore', 'statuscode': 409})
        else:
            sql = "INSERT INTO dinoProject.highscoreTable (sessionID, highscore) VALUES (%s, %s) ON DUPLICATE KEY UPDATE highscore = %s"
            values = (id, request.json["highscore"], request.json["highscore"])
            mydb.reconnect(attempts = 1 , delay = 0)
            mycursor.execute(sql, values)
            mydb.commit()
            mydb.disconnect()
            return jsonify({'status': 'updated highscore with id: ' + request.json["sessionID"], 'statuscode': 200})
        
class logTimestampsInDB(Resource):
    def put(self, id):
        print(request.json)
        if not request.json or not "timeData" in request.json or not "sessionID" in request.json:
            print("Error on Logging Timedata for Session: " + id)
        else:
            # Zeitdaten in File schreiben
            filename = id + '.txt'
            file = open(filename, "w+")
            
            timeData = request.json["timeData"]
            
            for time in timeData:
                file.write("Offset: %d ms\r\n" % time)
            
            # Durchschnitt bilden und hinzufügen
            average = sum(timeData) / len(timeData)
            file.write("Durchschnitt: %d ms\r\n" % average)
            
            file.close()
            
            # Blob erstellen
            binaryData = convertToBlob(filename)
            
            sql = "INSERT INTO dinoProject.logTable (sessionID, logFile) VALUES (%s, %s) ON DUPLICATE KEY UPDATE logFile = %s"
            values = (id, binaryData, binaryData)
            mydb.reconnect(attempts = 1 , delay = 0)
            mycursor.execute(sql, values)
            mydb.commit()
            mydb.disconnect()
            
def convertToBlob(filename):
    with open(filename, 'rb') as file:
                binaryData = file.read()
    return binaryData

In [ ]:
# Klassen an entsprechende URLs binden
api.add_resource(createSession, '/game')
api.add_resource(getOrAddHighscore, '/highscore/<id>')
api.add_resource(logTimestampsInDB, '/log/<id>')

# Run für Testzwecke, startet auf localhost:5000
if __name__ == "__main__":
    app.run(port=4000)

# Run für Dockercontainer, mit Portangabe
#if __name__ == "__main__":
 #   parser = optparse.OptionParser(usage="python server.py -p ")
  #  parser.add_option('-p', '--port', action='store', dest='port', help='The port to listen on.')
   # (args, _) = parser.parse_args()
    #if args.port == None:
     #   print "Missing required argument: -p/--port"
      #  sys.exit(1)
    #app.run(host='0.0.0.0', port=int(args.port), debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Sep/2019 15:59:58] "POST /game HTTP/1.1" 303 -
127.0.0.1 - - [05/Sep/2019 15:59:59] "GET /highscore/01083a942b7845579eb51f03296d5161 HTTP/1.1" 200 -


{'sessionID': '01083a942b7845579eb51f03296d5161', 'timeData': [2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 3, 2, 1, 1, 1, 1, 1]}


[2019-09-05 16:00:15,972] ERROR in app: Exception on /log/01083a942b7845579eb51f03296d5161 [PUT]
Traceback (most recent call last):
  File "/home/johannes/anaconda3/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 472, in cmd_query
    raw_as_string=raw_as_string)
_mysql_connector.MySQLInterfaceError: Cannot add or update a child row: a foreign key constraint fails (`dinoProject`.`logTable`, CONSTRAINT `logTable_ibfk_1` FOREIGN KEY (`sessionID`) REFERENCES `highscoreTable` (`sessionID`))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/johannes/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/johannes/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/johannes/anaconda3/lib/python3.7/site-packages/flask